In [2]:
import numpy as np
from sklearn.mixture import GaussianMixture
from tqdm import tqdm
from core.Skeleton import Skeleton

from core.MicronsCAVE import CAVE

class AxonModel(Skeleton):

    @staticmethod
    def create_axon_score_dict(axon_models, branches, collapsed=True):
        score_dict = {}
        for branch in tqdm(branches):
            cell_seq = branch.cell_id_sequence['collapsed'] if collapsed else branch.cell_id_sequence['raw']
            pos_seq = branch.syn_pos_sequence['collapsed'] if collapsed else branch.syn_pos_sequence['raw']
            branch_scores = []
            for cell_id in cell_seq:
                scores = axon_models[cell_id].gmm.score_samples(pos_seq)
                branch_scores.append(scores)
            score_dict[branch.branch_id] = branch_scores
        
        return score_dict

    def __init__(self, cell_info, syn_group, syn_k=8, soma_k=8, twig_length=4, single_syn_std=5):
        self.single_syn_std = single_syn_std
        super().__init__(cell_info, syn_group, syn_k, soma_k)
        self.smooth(twig_length, prune_unknown=False)
        self.fit_gmm()

    def fit_gmm(self, min_path_length=4):
        paths = self.get_paths(smoothed=True, duplicate_tail=True)
        all_positions = []
        all_means = []
        all_precisions = []
        for path in paths:
            path_positions = np.array([self.smooth_mst.nodes[node]['pos'] for node in path])
            mean = np.mean(path_positions, axis=0)
            if len(path_positions) >= min_path_length:
                precision = np.linalg.inv(np.cov(path_positions.T))
            elif len(path_positions) == 1:
                precision = np.diag(1/(np.array(3*[self.single_syn_std]))**2)
            else:
                precision = np.diag(1/np.var(path_positions, axis=0))
            
            precision[np.isinf(precision)] = 1/(self.single_syn_std**2)
            all_positions.append(path_positions)
            all_means.append(mean)
            all_precisions.append(precision)

        all_positions = np.concatenate(all_positions, axis=0)
        gmm = GaussianMixture(n_components=len(all_means), covariance_type='full', 
                means_init=np.array(all_means),
                precisions_init=np.array(all_precisions))
        gmm.fit(all_positions)
        self.gmm = gmm

In [10]:
import pandas as pd

exc_cells = pd.read_csv('data/exc_cells.csv')
client = CAVE()
syn_table = client.download_output_synapses(int(exc_cells.loc[2000].pt_root_id), cell_df=exc_cells)
print(len(syn_table))
syn_table.head()

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


49


,id,pre_pt_root_id,post_pt_root_id,size,cell_type_pre,cell_type_post,ctr_pt_x,ctr_pt_y,ctr_pt_z
0,33919160,864691136296739611,864691135677443844,11640,23P,Unknown,421.296,601.576,797.20
1,40159760,864691136296739611,864691136296739611,184,23P,23P,429.288,563.520,715.56
2,18024958,864691136296739611,864691135117914536,3356,23P,Unknown,360.600,616.120,779.68
3,53728319,864691136296739611,864691136118416408,4492,23P,5P-ET,469.480,623.096,772.28
4,43435552,864691136296739611,864691134744365116,5496,23P,Unknown,450.808,615.152,782.28


In [11]:
cell_info = exc_cells[exc_cells['pt_root_id']==exc_cells.loc[2000].pt_root_id]

axn = AxonModel(cell_info, syn_table, twig_length=4, single_syn_std=5)

In [12]:
axn.gmm

GaussianMixture(means_init=array([[440.75885714, 628.09257143, 759.20571429],
       [466.38628571, 657.36685714, 791.56      ],
       [455.74266667, 614.73866667, 713.97333333],
       [442.63961905, 533.92057143, 790.1352381 ],
       [390.326     , 648.972     , 759.556     ]]),
                n_components=5,
                precisions_init=array([[[ 8.07799973e-02, -3.04824564e-02, -1.58196729e-02],
        [-3.04824564e-02,  3.08874893e-01,  2.54398542e...
        [-1.35514533e-02,  6.69159632e-03, -1.16172635e-02],
        [ 2.74321027e-02, -1.16172635e-02,  2.69466671e-02]],

       [[ 2.67522835e-03, -9.63806732e-05,  5.71685804e-04],
        [-9.63806732e-05,  2.59945489e-04, -6.41536234e-05],
        [ 5.71685804e-04, -6.41536234e-05,  1.54347280e-03]],

       [[ 1.09500264e-02,  1.09515610e-03,  1.61983343e-02],
        [ 1.09515610e-03,  2.33199538e-03,  2.53286896e-03],
        [ 1.61983343e-02,  2.53286896e-03,  2.60438957e-02]]]))

In [13]:
axn.get_paths(smoothed=True, duplicate_tail=True)

{(4, 3, 5, 10, 45, 47, 37),
 (16,
  31,
  42,
  28,
  0,
  12,
  7,
  40,
  21,
  22,
  11,
  34,
  29,
  25,
  9,
  13,
  19,
  35,
  15,
  18,
  46),
 (30, 27, 32, 43, 41, 6, 48, 20, 2, 26),
 (30, 38, 24, 14, 17, 1),
 (36, 44, 39, 23, 8, 33, 30)}